Filtering criteria:

min.cells = 1000 per study
min.gene = 500
min.umi = 1000
max.MT = 30

In [ ]:
library(Seurat)
library(DoubletFinder)
library(HGNChelper)
library(tidyverse)
library(here)
source(here("notebooks/helper.R"))

In [10]:
files <- list.files(here("data/tnl"), full.names = TRUE, recursive = TRUE, pattern = 'filtered_feature_bc_matrix', ignore.case = TRUE)
names(files) <- str_extract(files, "(?<=/tnl/)[^/]+")

In [18]:
objs <- list()
for (i in 1:length(files)){
    mat <- Read10X_h5(files[[i]])
    obj <- CreateSeuratObject(counts = mat, min.features = 500, min.cells = 5, project = names(files[i]))
    objs <- append(objs, list(obj))
}

In [22]:
for(i in 1:length(objs)){
    objs[[i]][["percent.MT"]]  <- PercentageFeatureSet(objs[[i]], pattern = "^MT-") 
    objs[[i]] <- subset(objs[[i]], subset = percent.MT < 30 & nCount_RNA > 1000)
}

In [23]:
objs <- lapply(objs, seuPreProcess)

Normalizing layer: counts

Centering and scaling data matrix

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from pca_RNA_ to pcaRNA_”
Warning message:
“The following arguments are not used: force.recalc”
Warning message:
“The following arguments are not used: force.recalc”
Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”
Normalizing layer: counts

Centering and scaling data matrix

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from pca_RNA_ to pcaRNA_”
Warning message:
“The following arguments are not used: force.recalc”
Warning message:
“The following arguments are not used: force.recalc”
Normalizing layer: counts

Centering and s

In [ ]:
saveRDS(objs, here("data/tnl/tnl.rds"))

In [ ]:
bcmvn <- list()
pK <- list()
homotypic.prop <- list()
nExp_poi <- list()
nExp_poi.adj <- list()

# Estimated Doublet Rate for each dataset
edr <- estimateDoubletRate.DWM(seur.list = objs)/100 #use your own known EDR here

for(i in 1:length(objs)){
  cat(' ############################################\n',
      '### DoubletFinder for dataset number ', i, '###\n',
      '############################################\n')
  
  ## pK Identification (no ground-truth)
  bcmvn[[i]]<- paramSweep_v3(
    seu=objs[[i]],
    PCs = 1:objs[[i]]@reductions
n.pcs.used, 
    num.cores = 8
  ) %>% summarizeSweep(
    GT = FALSE
  ) %>% find.pK() 
  
  # Pull out max of bcmvn
  pK[[i]] <- as.numeric(as.character(bcmvn[[i]]
BCmetric==max(bcmvn[[i]]$BCmetric)])) # ugly, but functional...
  
  ## Homotypic Doublet Proportion Estimate
  homotypic.prop[[i]] <- modelHomotypic(objs[[i]]$seurat_clusters) 
  
  nExp_poi[[i]] <- round(edr[[i]]*length(colnames(objs[[i]])))  
  nExp_poi.adj[[i]] <- round(nExp_poi[[i]]*(1-homotypic.prop[[i]]))
}

In [ ]:
# Run DoubletFinder
for(i in 1:length(objs)){
  objs[[i]] <- 
    doubletFinder_V3.DWM_v2( # just changed it so the output metadata column name is customizable
      seu=objs[[i]], 
      PCs = 1:objs[[i]]@reductions
n.pcs.used, 
      pN = 0.25, #default value
      pK= pK[[i]], 
      nExp = nExp_poi.adj[[i]],  
      reuse.pANN = F, 
      classification.name='DF.individual', 
      pANN.name='DF.pANN.individual'
    )
}

In [ ]:
tnl <- merge(objs[[1]], y = unlist(objs[2:length(objs)]), add.cell.ids = labels, project = "tnl", merge.data = T)

In [ ]:
tnl <- subset(tnl, subset = DF.individual == 'Singlet')

In [ ]:
gene.names <- checkGeneSymbols(rownames(Wang2020_matrix), unmapped.as.na=FALSE)
rownames(Wang2020_matrix) <- make.unique(gene.names$Suggested.Symbol)